# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="https://github.com/MyaMjechal/nlp-a6-lets-talk-with-yourself-rag-chatbot/blob/main/images/RAG-process.png?raw=1" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [ ]:
# #langchain library
# !pip install langchain==0.0.350
# #LLM
# !pip install accelerate==0.25.0
# !pip install transformers==4.36.2
# !pip install bitsandbytes==0.41.2
# #Text Embedding
# !pip install sentence-transformers==2.2.2
# !pip install InstructorEmbedding==1.0.1
# #vectorstore
# !pip install pymupdf==1.23.8
# !pip install faiss-gpu==1.7.2
# !pip install faiss-cpu==1.7.4

In [17]:
#langchain library
!pip install langchain
#LLM
!pip install accelerate
!pip install transformers
!pip install bitsandbytes
#Text Embedding
!pip install sentence-transformers
!pip install InstructorEmbedding
#vectorstore
!pip install pymupdf
!pip install faiss-cpu
!pip install langchain-community
!pip install langchain-huggingface

  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.51-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.50-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.49-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.48-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.47-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_core-0.1.46-py3-none-any.whl.metadata (5.9 kB)
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.1.45-py3-none-any

In [3]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [4]:
from langchain import PromptTemplate

prompt_template = """
    I'm your friendly NLP chatbot named MJBot, here to to answer questions about Mya Mjechal myself based on my knowledge from my CV and personal data.
    The current year is 2025, and all answers should reflect this year unless otherwise specified.
    Whether you're curious about my education, work experience, or personal interests,
    I’ll provide accurate and gentle responses using the information I have.
    If I don't know something, I'll let you know kindly. Just let me know what you're wondering about, and I'll do my best to guide you through it!
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="I'm your friendly NLP chatbot named MJBot, here to to answer questions about Mya Mjechal myself based on my knowledge from my CV and personal data.\n    The current year is 2025, and all answers should reflect this year unless otherwise specified.\n    Whether you're curious about my education, work experience, or personal interests,\n    I’ll provide accurate and gentle responses using the information I have.\n    If I don't know something, I'll let you know kindly. Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:")

In [5]:
PROMPT.format(
    context = "My CV states that I graduated with a Bachelor’s degree in Computer Science from University of Information Technology.",
    question = "What is your highest level of education?"
)

"I'm your friendly NLP chatbot named MJBot, here to to answer questions about Mya Mjechal myself based on my knowledge from my CV and personal data.\n    The current year is 2025, and all answers should reflect this year unless otherwise specified.\n    Whether you're curious about my education, work experience, or personal interests,\n    I’ll provide accurate and gentle responses using the information I have.\n    If I don't know something, I'll let you know kindly. Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    My CV states that I graduated with a Bachelor’s degree in Computer Science from University of Information Technology.\n    Question: What is your highest level of education?\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code).
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [6]:
# from langchain.document_loaders import PyMuPDFLoader

# nlp_docs = '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf'

# loader = PyMuPDFLoader(nlp_docs)
# documents = loader.load()
from langchain.document_loaders import TextLoader

cv_file = 'data/MyaMjechal-CV.txt'
loader = TextLoader(cv_file)
documents = loader.load()

/usr/local/lib/python3.11/dist-packages/langchain_community/document_loaders/blob_loaders/file_system.py:5: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.document_loaders.blob_loaders.schema import Blob, BlobLoader
/usr/local/lib/python3.11/dist-packages/langchain_community/document_loaders/__init__.py:215: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for py

In [ ]:
# documents

In [7]:
len(documents)

1

In [8]:
documents[0]

Document(metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='Mya Mjechal\nFull Stack Developer\n\nPathum Thani, Thailand\nmyamjechal.mj@gmail.com\n\nBirthday: September 16, 1999\n\nI am a skilled developer with expertise in developing chatbots and dashboards, with additional skills in Natural Language Understanding (NLU) and Natural Language Processing (NLP) at EduTech Social Enterprise. I have experience as a freelancer, developing Odoo apps for local businesses, and a background in Software Development Management at Bliss Stock Co., Ltd. I am knowledgeable in Cluster Computing, Cloud Computing, Virtualization, Big Data Analysis, Networking, Web Programming, Blockchain, and IoT. I participated in the Huawei Cloud & AI contest, gaining experience in Data Mining, Image Processing, Data Prediction, and Cloud Usage. I implemented technologies in class projects, including OpenStack, Hyper-V Failover Clustering, Student Online Result System, and a Restaurant Guide Website. I have s

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [10]:
doc[1]

Document(metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='I am a skilled developer with expertise in developing chatbots and dashboards, with additional skills in Natural Language Understanding (NLU) and Natural Language Processing (NLP) at EduTech Social Enterprise. I have experience as a freelancer, developing Odoo apps for local businesses, and a background in Software Development Management at Bliss Stock Co., Ltd. I am knowledgeable in Cluster Computing, Cloud Computing, Virtualization, Big Data Analysis, Networking, Web Programming, Blockchain, and IoT. I participated in the Huawei Cloud & AI contest, gaining experience in Data Mining, Image Processing, Data Prediction, and Cloud Usage. I implemented technologies in class projects, including')

In [11]:
len(doc)

10

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [18]:
import torch
# from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"

embedding_model = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [19]:
#locate vectorstore
vector_path = '../vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [20]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'myamjechal_cv'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'cv' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [21]:
#calling vector from local
vector_path = '../vector-store'
db_file_name = 'myamjechal_cv'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'cv' #default index
)

In [22]:
#ready to use
retriever = vectordb.as_retriever()

In [23]:
retriever.get_relevant_documents("What is your name?")

<ipython-input-23-50d30f9d0dc1>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("What is your name?")


[Document(metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='Mya Mjechal\nFull Stack Developer\n\nPathum Thani, Thailand\nmyamjechal.mj@gmail.com\n\nBirthday: September 16, 1999'),
 Document(metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='PERSONAL INFORMATION\nAddress: Pathum Thani, Thailand\nNationality: Burmese\nDriving License: No\nHobbies: Reading, watching movies and series, coding, singing and listening to music\n\nLANGUAGES\nBurmese - Native\nEnglish - Fluent\nJapanese - N3\nThai - Beginner'),
 Document(metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='WORK EXPERIENCE\n\nJanuary 2025 – Present\nTechnical Support | intERLab (Asian Institute of Technology)\nAs a Freelance Technical Support Student Assistant at intERLab, AIT, I work on WordPress migration, ensuring smooth website transfers and functionality. I troubleshoot technical issues, assist with system configurations, and improve website performance.\n\nJanuary 2024 – December 2024\nSenior A

In [24]:
retriever.get_relevant_documents("How old are you?")

[Document(metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='ADDITIONAL INFORMATION\n\nAge: 25 years old (as of 2025)\n\nHighest Level of Education: Master of Science (Expected May 2025)\n\nMajor: Data Science and Artificial Intelligence\n\nWork Experience: Over 4 years in software development, full-stack development, and technical support\n\nIndustries: EduTech, IT Services, Software Development, Cloud Computing, AI/ML, and WordPress development\n\nCurrent Role: Freelance Technical Support at intERLab, focusing on WordPress migration, troubleshooting, and system configurations\n\nCore Beliefs on Technology: Technology should enhance accessibility, education, and opportunities while being ethically and culturally responsible.'),
 Document(metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='June 2022 – December 2023\nComputer Programmer | EduTech Social Enterprise, Yangon\nI built a chatbot and LMS dashboard, provided IT support, and shared knowledge with peers.\n\nSe

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [34]:
from langchain.memory import ChatMessageHistory

# Create a ChatMessageHistory
history = ChatMessageHistory()
history

/usr/local/lib/python3.11/dist-packages/pydantic/v1/validators.py:767: UserWarning: Mixing V1 and V2 models is not supported. `BaseMessage` is a V2 model.
  warn(f'Mixing V1 and V2 models is not supported. `{type_.__name__}` is a V2 model.', UserWarning)


RuntimeError: no validator found for <class 'langchain_core.messages.base.BaseMessage'>, see `arbitrary_types_allowed` in Config

In [ ]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [ ]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios.
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time.
- it only uses the last K interactions.
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available),
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [ ]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

In [ ]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = '../models/fastchat-t5-3b-v1.0/'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

2024-01-02 14:57:27.374664: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-02 14:57:27.392320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 14:57:27.392344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 14:57:27.392979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 14:57:27.396491: I tensorflow/core/platform/cpu_feature_guar

### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [ ]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [ ]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [ ]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [ ]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing Both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing Both of them',
 'text': '<pad> What  is  the  difference  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [ ]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7fefa8f7ec50>)), document_variable_name='context')

In [ ]:
query = "What is Transformers?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    formers are not based on recurrent connections (which can be hard to parallelize),
which means that transformers can be more efﬁcient to implement at scale.
Transformers map sequences of input vectors (x1,...,xn) to sequences of output
vectors (y1,...,yn) of the same length. Transformers are made up of stacks of trans-
former blocks, each of which is

{'input_documents': [Document(page_content='formers are not based on recurrent connections (which can be hard to parallelize),\nwhich means that transformers can be more efﬁcient to implement at scale.\nTransformers map sequences of input vectors (x1,...,xn) to sequences of output\nvectors (y1,...,yn) of the same length. Transformers are made up of stacks of trans-\nformer blocks, each of which is a multilayer network made by combining simple\nlinear layers, feedforward networks, and self-attention layers, the key innovation of\nself-attention\ntransformers. Self-attention allows a network to directly extract and use information\nfrom arbitrarily large contexts without the need to pass it through intermediate re-', metadata={'source': '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'file_path': '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf', 'page': 219, 'total_pages': 636, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator

In [ ]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). \n    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. \n    Whether it's about probabilistic models, language models, or any other related topic, \n    I'm here to help break down complex concepts into easy-to-understand explanations.\n    Just let me know what you're wondering about, and I'll do my best to guide you through it!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2Te

## 5. Chatbot

In [ ]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP). 
    If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have. 
    Whether it's about probabilistic models, language models, or any other related topic, 
    I'm here to help break down complex concepts into easy-to-understand explanations.
    Just let me know what you're wondering about, and I'll do my best to guide you through it!
    in this way are also called complementizers.
complementizer
Pronouns act as a shorthand for referring to an entity or event. Personal pro-
pronoun
nouns refer to persons or entities (you, she, I, it, me, etc.). Possessive pronouns are
forms of personal pronouns that indicate either actual possess

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': '<pad>  I  am  Chaky.\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include`< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.6\n Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol:\n< s>\n I  do  not  like  green  eggs  and  Sam\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include{< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.7\n Suppose  we  didn’t  use  the  end-symbol  .  Train ',
 'source_documents': [Document(page_content='in this way are also called complementizers.\ncomplementizer\nPronouns act as a shorthand for referring to an entity or event. Personal pro-\npronoun\nnouns refer to persons or entities (you, she, I, it, me, etc.). Possessive pronouns are\nforms of personal pro

In [ ]:
prompt_question = "What is the Transformers?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  I  am  Chaky.\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include`< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.6\n Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol:\n< s>\n I  do  not  like  green  eggs  and  Sam\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include{< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.7\n Suppose  we  didn’t  use  the  end-symbol  .  Train ')]
Follow Up Input: What is the Tran

{'question': 'What is the Transformers?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  I  am  Chaky.\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include`< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.6\n Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol:\n< s>\n I  do  not  like  green  eggs  and  Sam\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include{< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.7\n Suppose  we  didn’t  use  the  end-symbol  .  Train ')],
 'answer': '<pad>  transformers  are  a  type  of  neural  network  that  are  used  to  map  sequences  of  input  vectors  (x1,...,xn)  to  sequences  of  output  vectors  (y1,...,yn)  of  the  same  length.  They  are  made  up  of  stacks  of 

In [ ]:
prompt_question = "Is it a statistical model?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content='<pad>  I  am  Chaky.\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include`< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.6\n Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol:\n< s>\n I  do  not  like  green  eggs  and  Sam\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include{< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.7\n Suppose  we  didn’t  use  the  end-symbol  .  Train '), HumanMessage(content='What is the

{'question': 'Is it a statistical model?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content='<pad>  I  am  Chaky.\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include`< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.6\n Suppose  we  didn’t  use  the  end-symbol  .  Train  an  unsmoothed  bigram  grammar  on  the  following  training  corpus  without  using  the  end-symbol:\n< s>\n I  do  not  like  green  eggs  and  Sam\n Using  a  bigram  language  model  with  add-one  smoothing,  what  is  P(Sam  |  am)?  Include{< s>` and` am` in  your  counts  just  like  any  other  token.\n 3.7\n Suppose  we  didn’t  use  the  end-symbol  .  Train '),
  HumanMessage(content='What is the Transformers?'),
  AIMessage(content='<pad>  transformers  are  a  type  of  neural  network  that  are  used  to  map  sequences  of  input  vectors  (x1,...,xn)  to  sequences  of  output  vectors  (y1,...,yn)